In [22]:
# Importing some shizzle.

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import time
import requests
import datetime
import sys
import os
import pathlib
import seaborn as sns
from sklearn.metrics import roc_auc_score
import math
import json
import glob
import xlrd3 as xlrd
import gzip
from utilities import perform_stats, calc_z_scores,\
    get_enrichr_results, plot_results,\
    plot_results_in_singleplot, plot_results_in_singleplot_iteractive_legend,\
    count_words_in_df, plot_count_results, split_and_count_words

# Defining some import and export locations
location = 'rjsietsma'
read_loc = '/home/'+location+'/shared/'
data_expor_loc = '/home/'+location+'/Documents/School/Master_DSLS/Final_Thesis/Past_initial_data/'
img_output_dir = '/home/'+location+'/PycharmProjects/dsls_master_thesis/side_scripts/output_img/'

In [23]:
# Some bokeh stuff

from bokeh.plotting import figure, ColumnDataSource, output_file, show, save
from bokeh.models import HoverTool, WheelZoomTool, PanTool,\
    BoxZoomTool, ResetTool, TapTool, SaveTool, FactorRange, Legend
from bokeh.palettes import inferno, Spectral5, viridis, Colorblind
from bokeh.transform import factor_cmap

Sprint 28-04-2020 - 19-05-2020 (?)
=====================

- What I want to do:
  1. [Focus on fixing missing CAPICE precomputed scores.](#Fixing-CAPICE-precomputed-scores) __Done__
      - _Note: Find out how the script of Shuang works on a smaller scale, get an example where a CAPICE score is missing._
  2. [Normalize BM_ratio](#Normalize-BM_ratio) for gene length, so $\frac{n_{malignant}}{n_{benign}}$ per 1000 bases in a gene, instead of [$\frac{n_{malign}}{n_{benign}} \times n_{total}$](output_img/bm_ratio.png "BM_ratio").
  3. _(questionable)_ Fix PCA by identifying categorical features _(Questionable because should I still put time and focus in on the input data?)_
      - _Ask Shuang, Joeri and Krista in the PRU 28-04-2020 @ 13:00-14:00_
  4. [See what is required](#Required-for-data-sources) for the [data sources](https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit) to be implemented into the input data.
      - _Note: as this is research, I can use sources that only have precomputed scores available, but just to keep in mind that I should mention that in the discussion._

- Notes:
  - If I find myself unfocussed:
      - Win a game of [solitaire](https://play.google.com/store/apps/details?id=com.lemongame.klondike.solitaire&hl=nl) (game is rigged, not every match is win-able).
      - Clean out my closet (it's messy, maybe to clean up the mess inside my head too).
      - Go for a walk around the neighbourhood.

<br/><br/>
<br/><br/>
### Fixing CAPICE precomputed scores

Problem: CAPICE precomputed file of all single nucleotide level variants contains caps.

Solution: Fix the underlying problem that causes these gaps.

Required: 
- Script to make and batch the CAPICE computing __(positive check)__
- Access to GCC cluster __(positive check)__
- Knowledge of job scheduler on GCC cluster __(positive check)__
- Smaller batches of SNV files to run locally  __(positive check)__

Possible cause:
- Incorrect batching on tabix.

31-04-2020 Problem:
- Got it working on Gearshift, but with a problem:
    - AttributeError: Can't get attribute 'DeprecationDict' on <module 'sklearn.utils.deprecation' from '/apps/software/PythonPlus/3.7.4-foss-2018b-v20.02.1/lib/python3.7/site-packages/sklearn/utils/deprecation.py'>
- Which let's me to beleive that the model was trained in a previous version of sklearn.

<br/><br/>

- Update 10:51 : Model compatibility issue. Might have to retrain CAPICE on Gearshift using the latest xgboost, sklearn, scikit-learn, numpy and pandas just to be safe.

- Update 12:01 : Managed to get it working using python3.6 on calculon, essentially downgrading my own script (I only used python3.7 for F-strings)

- Update 15:55 : Found a very weird bug in the chunking. Seems like the script doesn't like to start at 0 with skiprows and 1000 nrows, solution: nrows - 2 when first iteration, then + 1, then start += nrows + 1...

- Update 04-05-2020 : Found out that the first iteration, an overhead of 2 variants was present. Adjusted the start with 2 to compensate. Seemed fine in the first 400000 CADD entries. (no duplicates nor missed entries). __Started full processing__

- Update 04-05-2020 : 
    - Add functionality to continue where left off if it crashes.
    - Make dummy script to check for duplicates or if the files are fully complete.
    
- Update 06-05-2020 : 
    - Functionaility to continue where left off is required. After 40 hours, it's still processing chromosome 1 after 7 million processed variants. 
        - Ideas: write json file with start and stop each iteration once that part is processed and added to the larger files. Should overwrite.
        - As for now, since this idea is not implemented yet, count the number of rows in the already processed files and set that as start.

- Update 11-05-2020:
    - The 4 days have ended, chromosome 1 has not been finished yet.
    - Output file of chromosome 1 has been renamed to a .tsv file and gzipped (saving about 5 gb of storage).
    - Adjustments to the original script have to be made.
   

#### Version 2.0

- Added gzip output by default
- Currently no function yet to continue where left off.
    - Ideas:
        - for line in gzip.open(file.gz): nlines+=1 (to trace back where was left off)
            - Do not know if this lead to any complications of duplicated entries. Saving the previous results in the RAM memory might solve this to compare to.
    - Then in the future:
        - Save a json of start and batch size each iteration to mark where the file was left of.

- Update 15-05-2020:
    - Program did not load in the previous results correctly, resulting in 12 duplicates (111400001-111400012) (including start and stop). Discuss with Joeri and Shuang what to do. It takes roughly 200 MB of ram to load in 2.1 million processed variants, calculating this for 111.4 million variants (that's how much there is in the file now), would come down to ~11GB of ram usage.

- Update 18-05-2020:
    - Script to remove duplicates from the CAPICE processed file has been run and duplicates have been removed.
    - First test runs have been ran, only for me to realised I messed up. I kept copying the main class, instead of the calculate_capice class. Final release should be on this date.
    

<br/><br/>
<br/><br/>
### Normalize BM_ratio

Problem: Larger genes have more benign and malignant samples, because they are larger and thus the chanses of variation on the reference read is higher.

Solution: Make a $n_{malignant}$ and $n_{benign}$ per $x$ bases in a gene, where {$x \in \mathbb{N}\ | x \geq 1 | x \leq 10000$}

<br/><br/>
<br/><br/>
### Required for data sources

Problem: Now that a lot of data sources have been found, annotation and merging has to be performed in order to use it as training data.
- _Additional problem: once a source has been proven beneficial, it has te be implemented to be annotated before capice fits and predicts a new sample._

Solution: Find out, feature by feature if genes that have a low AUC improve *__and that genes that have a high AUC do not perform worse__*

Required:
- Storage 
    - On GCC cluster for precomputed score files.
    - Locally IF file isn't that big or an API can be used.
- Data sources __([positive check](https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit))__
- Script to annotate / merge


      

# CTEPG (Catching The Elusive Predictable Genes) 28-04-2020 - 04-05-2020
<br/><br/>

What have I done:

- [Enrichr Genes < 0.6](https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=b7a959357214069d1905719220ed3e3b)
- [Enrichr Genes < 0.7](https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=7d252a3e1df275859228fd562e964344)
    - For reference: [Enrichr Genes < 0.85](https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=60c74741c967c25331419bd2b5e7183e)
- [Finished investigating initial data sources](https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit)

<br/><br/>
<br/><br/>

What I'm planning to do this week:

- [Fixing missing values in precomputed CAPICE scoring file](#Fixing-CAPICE-precomputed-scores). __Done and not done, script has to have some additonal features__
    - If stuck, do not keep walking into the wall, continue below and contact Shuang and Joeri
- [Run enrichr on increasing cutoffs in AUC (.5, .6, .7 etc.)](#Enrichr-automated-API). __Done__
    - Read into the [Enrichr API documentation](https://amp.pharm.mssm.edu/Enrichr/help#api)
- Mail Tsjerk that a meeting is apreciated. __Done, no response yet__
- Make [suggested plot](https://docs.google.com/presentation/d/1_woKoD9nqWnxueyFYfwyX6B4owg8vYF7rw_nKpMPux4/edit#slide=id.p). __Done__
    - Select from enrichr all the genes involved in (lipid) metabolism and see if this explains all / most of the bad performing genes.
- Make model on bad performing genes and investigate further.
- Start investigating what is required for the data sources to be used in the training phase.
- Look into the genes that do perform well without a CGD category, also look into the 1/2 genes that do not perform well __with__ a CGD category
- _Optionally_ :
    - Create outline for article.
    - Adjust BM_ratio plot to normalize for gene length.
    - Get the UMCG gene panel lists into python
        - Then: investigate which gene panel is performing the worst.
        - It would be nice to have this on the side for diagnostics, do __not__ focus on this
    - Fix PCA.
    - Find possible way to annotate genes for co-factors, sub-family etc.
<br/><br/>

# CTEPG (Catching The Elusive Predictable Genes) 04-05-2020 - 11-05-2020
<br/><br/>

What have I done:
- [Fixing missing values in precomputed CAPICE scoring file](#Fixing-CAPICE-precomputed-scores).
- [Run enrichr on increasing cutoffs in AUC (.5, .6, .7 etc.)](#Enrichr-automated-API).
- Make [suggested plot](https://docs.google.com/presentation/d/1_woKoD9nqWnxueyFYfwyX6B4owg8vYF7rw_nKpMPux4/edit#slide=id.p).
<br/><br/>

Planned for this week:
- Release precomputed CAPICE script v2.0
- Investigate Bokeh plots on commonalities between Adjusted P-value of 0-0.2 and AUC of 0.5-~0.75 and try to find features / links that would be beneficial to train a separate model on. (or prehaps just train a new model without features and see if performance improves)
    - Ion transport / transmembrane related genes seems to perform bad. 
    - Find commonalities between y-axis 0-0.2 and x-axis 0-0.7
    - Make model on bad performing genes and investigate further.
        - Make plots combined of all sources, but color them by source.
- Start investigating what is required for the data sources to be used in the training phase.
- Look into the genes that do perform well without a CGD category, also look into the 1/2 genes that do not perform well __with__ a CGD category
- _Optionally_ :
    - Create outline for article.
        - For static plots: make labels on each point with the term name and the number of genes in (n=x) behind that.
    - Adjust BM_ratio plot to normalize for gene length.
    - Get the UMCG gene panel lists into python (calculate average AUC for each panel, take the worst performing onces and retain).
        - Then: investigate which gene panel is performing the worst.
        - It would be nice to have this on the side for diagnostics, do __not__ focus on this
    - Find possible way to annotate genes for co-factors, sub-family etc.
    - Interactive plot of all variants labeled as benign or pathogenic with their capice score (y-axis) and location (x-axis), to show the variant itself.
<br/><br/>

# CTEPG (Catching The Elusive Predictable Genes) 11-05-2020 - 18-05-2020
<br/><br/>

What have I done:
- [Fixing missing values in precomputed CAPICE scoring file](#Fixing-CAPICE-precomputed-scores).
- Make more bokeh plots on enrichr results
    - ~[Investigate correlations between AUC's and Adjusted P-value](#Correlation-analysis-Enrichr-results).  (spearman / pearson, preverably spearman). Keep in mind that the distribution is not linear, but more like a normal distribution like.~
        - ~Make legend interactive to confirm normal distribution like.~
    - Chromosome X seems enriched for higher AUC thresholds. Bias in the source data? (Put in discussion). Lot's of developmental delay reseach is linked / correlated to chromosome X.

Planned for this week:
- ~Release precomputed CAPICE script v2.0.~
- ~Run correlation test for all possible sources on Enrichr to maximize chanses of finding stuff.~
    - ~Look into a good way to get all possible results from the enrichr API.~ __Not possible.__
- ~Escalate mail contact with Tsjerk to Martijn and Fenna.~
    - __Unneeded, Tsjerk replied.__
- ~Investigate Bokeh plots on commonalities between Adjusted P-value of 0-0.2 and AUC of 0.5-0.75 and try to find features / links that would be beneficial to train a separate model on.~ (or prehaps just train a new model without features and see if performance improves)
    - Ion transport / transmembrane related genes seems to perform bad. 
    - ~Find commonalities between y-axis 0-0.2 and x-axis 0-0.7~
    - __Make model on bad performing genes and investigate further.__
        - ~Make plots combined of all sources, but color them by source.~
- Use regex `x[~x['INFO'].str.contains('CLNSIG=Uncertain_significance')]` to find all variants within the clinvar database that are not VUS'es
- Start investigating what is required for the data sources to be used in the training phase.
- Look into the genes that do perform well without a CGD category, also look into the 1/2 genes that do not perform well __with__ a CGD category
- _Optionally_ :
    - __Create outline for article.__
        - For static plots: make labels on each point with the term name and the number of genes in (n=x) behind that.
        - For first concept, it's due date is the last week of the resit exams on the 6th of July.
    - Adjust BM_ratio plot to normalize for gene length.
    - Get the UMCG gene panel lists into python (calculate average AUC for each panel, take the worst performing onces and retain).
        - Then: investigate which gene panel is performing the worst.
        - It would be nice to have this on the side for diagnostics, do __not__ focus on this
    - Find possible way to annotate genes for co-factors, sub-family etc.
    - Interactive plot of all variants labeled as benign or pathogenic with their capice score (y-axis) and location (x-axis), to show the variant itself.
<br/><br/>
    
## For after I graduate:
<br/><br/>
Make a script that for every gene, for every gene makes a plot like [these](https://molgenis26.gcc.rug.nl/downloads/capice/plots/), but then interactive that if you hover over a point you see it's chromosomal location, ref and alt and maybe an RS number?

- If input of CAPICE is not CADD annotated, annotate in capice locally and only return the capice scores.
 <br/><br/>
 
 # CTEPG (Catching The Elusive Predictable Genes) 02-06-2020 - 15-06-2020
<br/><br/>

What have I done:
- Get the UMCG gene panel lists into python (calculate average AUC for each panel, take the worst performing onces and retain).
     - Then: investigate which gene panel is performing the worst.
        - It would be nice to have this on the side for diagnostics, do __not__ focus on this
- Finished investigating Enrichr pathways. Only a couple are upregulated significantly when the AUC gets higher.

Planned for this week:
- ~Finish investigating Enrichr results.~
    - ~Make [suggested plots](https://docs.google.com/presentation/d/1e4zBxhPAGORXYJG0bulR1ADb68haa58v-agkFcv_cxQ/edit#slide=id.p).~
        - ~Plot 1: Color by main category and plot each panel average AUC in 1 bar plot to see if main categories cluster low / high.~
        - ~Plot 2: Compare all genes in 1 main category to all other categories independently P-values using the wilcoxon test. Play around with the test to see if higher / lower / other is applicable.~
- Make model on bad performing genes and investigate further.
- ~__Think about the following:__~
    - ~Work through the summer holiday as extra time to do research, potentially ending up with something amazing.~
    - ~Rush to get article / report finished, have summer holidays 'off' to do other school related stuff and relax a bit, potentially missing something amazing.~
- ~Make plot of Hart- en vaatziekten vs all other panels combined (minus 5GPM)~
- ~Make plot of Hart- en vaatziekten vs 5GPM~
- ~How many variants do I have in Hart- en vaatziekten, dyslipidimie (total)?~
    - If not enough variants are found (benign, malignant or total), look into [this](https://databases.lovd.nl/shared/variants/).
        - Usefull, [convert to vcf](http://grch37.ensembl.org/Homo_sapiens/Tools/VEP).
- Usefull, [convert 37 to 38](https://genome.ucsc.edu/cgi-bin/hgLiftOver).
- Start investigating what is required for the data sources to be used in the training phase.
- Look into the genes that do perform well without a CGD category, also look into the 1/2 genes that do not perform well __with__ a CGD category.
- Create outline for article.
     - For static plots: make labels on each point with the term name and the number of genes in (n=x) behind that.
     - For first concept, it's due date is the last week of the resit exams on the 6th of July.
- _Optionally_ :
    - Adjust BM_ratio plot to normalize for gene length.
    - Find possible way to annotate genes for co-factors, sub-family etc.
    - Interactive plot of all variants labeled as benign or pathogenic with their capice score (y-axis) and location (x-axis), to show the variant itself.
<br/><br/>
 
 # CTEPG (Catching The Elusive Predictable Genes) 15-06-2020 - 22-06-2020
<br/><br/>

What have I done:


Planned for this week:
- Make model on bad performing genes and investigate further.
    - At first, make models of [the original training data](https://zenodo.org/record/3564757#.XudiZ2ixUUE), models of various gene panels (but keep the variants of those panels within the bigger model). See if they improve and how much.
    - Then make a single model of the original training data for CADD 1.6 annotations.
    - Then make multiple models of the original training data for CADD 1.6 annotations.
    - Investigate improvements.
    - Then choose between:
        - newer / larger dataset or 
        - Custom features that might be relevant to the cause (prefered)
- Make single plot of panel vs all others combined (except for 5GPM).
- Usefull, [convert 37 to 38](https://genome.ucsc.edu/cgi-bin/hgLiftOver).
- Start investigating what is required for the data sources to be used in the training phase.
- Create outline for article.
     - For static plots: make labels on each point with the term name and the number of genes in (n=x) behind that.
     - For first concept, it's due date is the last week of the resit exams on the 6th of July.
- _Optionally_ :
    - Find possible way to annotate genes for co-factors, sub-family etc.
    - Interactive plot of all variants labeled as benign or pathogenic with their capice score (y-axis) and location (x-axis), to show the variant itself.
<br/><br/>



  
    

# Current Tickets:
<br/><br/>

## As student / Robert I want to map requirements for SNV and gene features.

- Content:

Overview of features has now been collected and processed (https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit). In order to use these features, knowledge has to be obtained for the requirements on said features including, but not limited to merging to the trianing dataset and storage.

- Acceptance criterea:
    - Does not take up terrabytes of storage.
        - _If it does, data sources might have to be merged 1 by 1, removing the source after the merge._
    - Can be merged without 3rd party software (tabix, python etc. allowed).
    - Data can easily be downloaded.

<br/><br/>

## ~As student / Robert I want to find commonalities between bad performing genes in CAPICE~

- Content:

The enrichr tool has proven to be quite beneficial to understanding a set of bad performing genes. Further processing of these enrichr results is required in order to fully understand the underlying mechanics and commonalities that these bad performing genes share.

- Acceptance criterea:
    - Enrichr is performed on multiple gene sets of different AUC thresholds (0.5, 0.6, 0.7 etc.).
    - Results of various data sources (kegg, bioplanet, wikipathways etc.) have been obtained in an automated way using the Enrichr API.
    - Results from the Enrichr API have been processed in an understandable plot, an mockup can be found [here](https://docs.google.com/presentation/d/1_woKoD9nqWnxueyFYfwyX6B4owg8vYF7rw_nKpMPux4/edit#slide=id.p).

<br/><br/>

## As CAPICE user I want to use the best possible performing model

- Prerequisites:

    - As Bioinformatician I want to update CAPICE by preprocessing train test datasets to include all features that CAPICE learns on.
    - As researcher I want to know in detail how sample weights are applied.
    

- Content:

Even though CAPICE performs really well, there is room for improvement. Subsetting genepanels in the model training and predicting phase might prove beneficial to the performance of CAPICE. Furthermore, the model must be updated a bit to fit certain quality of life improvements (see acceptance criterea).

- Acceptance criterea:
    - Performance of all gene panels must not worsen.
    - Subsetting the variants of the different panels must happen within the model.
    - The model must be able to run on python 3.6 and python 3.8 for future proofing.
    - Performance must increase with as least additional features as possible.
        - For this, sample size must remain within acceptable boundaries. (acceptable is discussable)
        - Additional features shall focus primarily on gene level to limit storage and processing stress.
        - When variant level features shall be added, this shall be performed one datasource at a time, again to limit stress.
        - Additional features must be beneficial for all gene panels.
            - Certain features might prove beneficial to a certain gene panel, but not to others, could indicate that the cause of worse performance for that certain panel lies within that feature.
    - The model will be created for genome build 37 in the first place, but in such a way that build 38 can easily be added as supported.
    - The model must support CADD 1.4 and CADD 1.6
        - CADD 1.5 is basically CADD 1.4 but with fixed GERP annotations for build 38.
    - The model must use precomputed scores first. When no precomputed score is found for the variant, it shall be computed.
    

# Enrichr automated API

In [24]:
enrichr_url = 'http://amp.pharm.mssm.edu/Enrichr/'

In [25]:
# Load in initial data
data = pd.read_csv(read_loc+'optimal_f1_full_ds_v2.csv', header=0)
data.rename(columns={
    'default_auc': 'auc'
}, inplace=True)
data.sort_values(by='default_f1', ascending=False)

,gene,auc,default_f1,default_recall,default_fpr,default_spec,optimal_c,optimal_auc,optimal_f1,optimal_recall,optimal_fpr,optimal_spec,n_tot,n_benign,n_malign
2775,PRKN,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,11,7,4
1659,BUB1B,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,73,66,7
2102,GALNT3,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,12,10,2
1655,PNPLA1,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,0.8,0.2,0.8,31,26,5
2782,TMEM80,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,5,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2625,C10orf2,0.469697,0.0,0.0,1.0,0.0,0.020,0.469697,0.000000,0.0,1.0,0.0,133,132,1
2330,SPTLC2,0.455157,0.0,0.0,1.0,0.0,0.011,0.750374,0.170213,0.0,1.0,0.0,229,223,6
2615,PLCD1,0.500000,0.0,0.0,1.0,0.0,0.020,0.500000,0.000000,0.0,1.0,0.0,8,7,1
2611,UBE2A,0.500000,0.0,0.0,1.0,0.0,0.020,0.500000,0.000000,0.0,1.0,0.0,3,2,1


In [26]:
data.shape[0]

2807

In [27]:
# data[['n_tot', 'n_benign', 'n_malign']].sum()

In [28]:
# Make a dictionary of lists of genes of interest with as key the 0.x threshold

auc_dict = {}
thresholds = np.arange(0.5, 1.01, 0.05).round(decimals=2)
for t in thresholds:
    list_of_genes = data[data['auc'] < t]['gene'].tolist()
    auc_dict[t.astype('str')] = '\n'.join(list_of_genes)
    del list_of_genes
del thresholds

# Save it to a file so I don't bombard the Enrichr API with requests
if not os.path.isfile('./user_id_list.json'):
    file = open('./user_id_list.json', 'w')
    file.close()

with open('./user_id_list.json') as json_file:
    try:
        user_list_ids = json.load(json_file)
        json_was_empty = False
    except json.JSONDecodeError:
        user_list_ids = {}
        json_was_empty = True

In [29]:
# Get list ID's for the different AUC's

for threshold in auc_dict.keys():
    if threshold not in user_list_ids.keys():
        description = f'CAPICE performance of genes with an AUC lower than {threshold}'
        payload = {
            'list': (None, auc_dict[threshold]),
            'description': (None, description)
        }
        response = requests.post(enrichr_url + '/addList', files=payload)
        if not response.ok:
            print(f'Something went wrong with {threshold}!')
        else:
            enrichr_addlist_response = json.loads(response.text)
            user_list_ids[threshold] = enrichr_addlist_response['userListId']

# Write it to a file in case I didn't have any.
if json_was_empty:
    with open('./user_id_list.json', 'w') as json_file:
        json.dump(user_list_ids, json_file)

In [30]:
# Now to get the enrichment results.

enrichr_sources_oi = ['BioPlanet_2019', 'WikiPathways_2019_Human', 'KEGG_2019_Human', 'Elsevier_Pathway_Collection',
                     'GO_Biological_Process_2018','Human_Phenotype_Ontology','Jensen_DISEASES',
                     'PheWeb_2019', 'ClinVar_2019', 'GWAS_Catalog_2019', 'DisGeNET','OMIM_Disease','OMIM_Expanded',
                     'Human_Gene_Atlas', 'ARCHS4_Tissues',
                     'HMDB_Metabolites','Chromosome_Location_hg19']


In [31]:
output_source_link = {}
for source in enrichr_sources_oi:
    output_filename = source + '.csv'
    output_source_link[source] = output_filename

In [32]:
glob_files_present = glob.glob('./EnrichrAPIResults/*.csv')
list_of_files_present = []
if len(glob_files_present) > 0:
    for file in glob_files_present:
        list_of_files_present.append(file.split('/')[-1])

In [33]:
sources_required = []
for source, output_filename in output_source_link.items():
    if output_filename not in list_of_files_present:
        sources_required.append(source)

In [34]:
sources_required

['BioPlanet_2019',
 'WikiPathways_2019_Human',
 'KEGG_2019_Human',
 'Elsevier_Pathway_Collection',
 'GO_Biological_Process_2018',
 'Human_Phenotype_Ontology',
 'Jensen_DISEASES',
 'PheWeb_2019',
 'ClinVar_2019',
 'GWAS_Catalog_2019',
 'DisGeNET',
 'OMIM_Disease',
 'OMIM_Expanded',
 'Human_Gene_Atlas',
 'ARCHS4_Tissues',
 'HMDB_Metabolites',
 'Chromosome_Location_hg19']

In [35]:
get_enrichr_results(sources_required, user_list_ids, 20)

Working on source: BioPlanet_2019
Sources left: ['WikiPathways_2019_Human', 'KEGG_2019_Human', 'Elsevier_Pathway_Collection', 'GO_Biological_Process_2018', 'Human_Phenotype_Ontology', 'Jensen_DISEASES', 'PheWeb_2019', 'ClinVar_2019', 'GWAS_Catalog_2019', 'DisGeNET', 'OMIM_Disease', 'OMIM_Expanded', 'Human_Gene_Atlas', 'ARCHS4_Tissues', 'HMDB_Metabolites', 'Chromosome_Location_hg19']
	Currently working on threshold: 0.5


/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/pandas/core/frame.py:4160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


	Currently working on threshold: 0.55
	Currently working on threshold: 0.6
	Currently working on threshold: 0.65
	Currently working on threshold: 0.7
	Currently working on threshold: 0.75
	Currently working on threshold: 0.8
	Currently working on threshold: 0.85
	Currently working on threshold: 0.9
	Currently working on threshold: 0.95
	Currently working on threshold: 1.0
	Source BioPlanet_2019 has been exported to csv (
		./EnrichrAPIResults/BioPlanet_2019.csv
	)
Working on source: WikiPathways_2019_Human
Sources left: ['KEGG_2019_Human', 'Elsevier_Pathway_Collection', 'GO_Biological_Process_2018', 'Human_Phenotype_Ontology', 'Jensen_DISEASES', 'PheWeb_2019', 'ClinVar_2019', 'GWAS_Catalog_2019', 'DisGeNET', 'OMIM_Disease', 'OMIM_Expanded', 'Human_Gene_Atlas', 'ARCHS4_Tissues', 'HMDB_Metabolites', 'Chromosome_Location_hg19']
	Currently working on threshold: 0.5
	Currently working on threshold: 0.55
	Currently working on threshold: 0.6
	Currently working on threshold: 0.65
	Currently wo

	Currently working on threshold: 0.55
	Currently working on threshold: 0.6
	Currently working on threshold: 0.65
	Currently working on threshold: 0.7
	Currently working on threshold: 0.75
	Currently working on threshold: 0.8
	Currently working on threshold: 0.85
	Currently working on threshold: 0.9
	Currently working on threshold: 0.95
	Currently working on threshold: 1.0
	Source OMIM_Disease has been exported to csv (
		./EnrichrAPIResults/OMIM_Disease.csv
	)
Working on source: OMIM_Expanded
Sources left: ['Human_Gene_Atlas', 'ARCHS4_Tissues', 'HMDB_Metabolites', 'Chromosome_Location_hg19']
	Currently working on threshold: 0.5
	Currently working on threshold: 0.55
	Currently working on threshold: 0.6
	Currently working on threshold: 0.65
	Currently working on threshold: 0.7
	Currently working on threshold: 0.75
	Currently working on threshold: 0.8
	Currently working on threshold: 0.85
	Currently working on threshold: 0.9
	Currently working on threshold: 0.95
	Currently working on thre

In [36]:
enrichr_data_loc = './EnrichrAPIResults/'

enrichr_dataset = {}

for source in enrichr_sources_oi:
    dataset = enrichr_data_loc + source + '.csv'
    enrichr_dataset[source] = pd.read_csv(dataset)

In [37]:
enrichr_dataset['BioPlanet_2019'].head()

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,3,Activation of TRKA receptors,NTRK2,1.0,BioPlanet_2019
1,0.5,4,MAP kinase pathway regulation through dual spe...,DUSP6,1.0,BioPlanet_2019
2,0.5,5,ERK inactivation,DUSP6,1.0,BioPlanet_2019
3,0.5,7,Nucleotide-like (purinergic) G-protein coupled...,P2RY12,1.0,BioPlanet_2019
4,0.5,8,Gap junction assembly,GJB3,1.0,BioPlanet_2019


In [38]:
for key in enrichr_dataset.keys():
    plot = plot_results(enrichr_dataset[key], 10)
    save(plot)

In [39]:
combined_df = pd.DataFrame(columns=enrichr_dataset['BioPlanet_2019'].columns)
for key, value in enrichr_dataset.items():
    subdf = pd.DataFrame(columns=combined_df.columns)
    for auc in value['AUC'].unique():
        subdf = subdf.append(value[value['AUC'] == auc][:10], ignore_index=True)
    combined_df = combined_df.append(subdf, ignore_index=True)
combined_df

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,3,Activation of TRKA receptors,NTRK2,1.0,BioPlanet_2019
1,0.5,4,MAP kinase pathway regulation through dual spe...,DUSP6,1.0,BioPlanet_2019
2,0.5,5,ERK inactivation,DUSP6,1.0,BioPlanet_2019
3,0.5,7,Nucleotide-like (purinergic) G-protein coupled...,P2RY12,1.0,BioPlanet_2019
4,0.5,8,Gap junction assembly,GJB3,1.0,BioPlanet_2019
...,...,...,...,...,...,...
1804,1.0,8,chr16,"CTCF, LITAF, PKD1, SALL1, CDH1, TUBB3, CHMP1A,...",1.0,Chromosome_Location_hg19
1805,1.0,6,chr6 mcf hap5,"TNXB, CYP21A2, TUBB, COL11A2, SKIV2L, VARS2, C...",1.0,Chromosome_Location_hg19
1806,1.0,9,chr15,"GABRB3, MTFMT, HEXA, ETFA, UBE3A, CHD2, NR2E3,...",1.0,Chromosome_Location_hg19
1807,1.0,7,chr6 dbb hap3,"TNXB, CYP21A2, TUBB, COL11A2, SKIV2L, VARS2, C...",1.0,Chromosome_Location_hg19


In [40]:
combined_df.head(20)

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.50,3,Activation of TRKA receptors,NTRK2,1.0,BioPlanet_2019
1,0.50,4,MAP kinase pathway regulation through dual spe...,DUSP6,1.0,BioPlanet_2019
2,0.50,5,ERK inactivation,DUSP6,1.0,BioPlanet_2019
3,0.50,7,Nucleotide-like (purinergic) G-protein coupled...,P2RY12,1.0,BioPlanet_2019
4,0.50,8,Gap junction assembly,GJB3,1.0,BioPlanet_2019
5,0.50,9,Interleukin-11 pathway,DUSP6,1.0,BioPlanet_2019
6,0.50,10,ADP signalling through P2Y purinoceptor 12,P2RY12,1.0,BioPlanet_2019
7,0.50,11,Gap junction trafficking and regulation,GJB3,1.0,BioPlanet_2019
8,0.50,12,Nuclear events mediated by MAP kinases,DUSP6,1.0,BioPlanet_2019
9,0.50,13,Signal amplification,P2RY12,1.0,BioPlanet_2019


In [41]:
plot = plot_results_in_singleplot_iteractive_legend(combined_df)
save(plot)

'/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/combined_enrichr_plot_results_interactive_legend.html'

In [42]:
plot = plot_results_in_singleplot(combined_df)
save(plot)

'/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/combined_enrichr_plot_results.html'

## Interpretation
It seems that a normal distribution like pattern is formed with a steeper slope after the maximum, starting at rougly AUC of 0.3 and ending at roughly 0.8.
<br/><br/>
- DisGeNET seems to outline an interesting picture with the top 10 term names:
    - For it seems that in the really bad performing genes the main cause could be infant development related. Later on, more neurological causes are identified.
        - Neurological seems clear to me... There is still so much unknown about the human mind.
        - Infant development related is a bit less clear, since there is a growing amount of research towards this infant development.
- GO_biological_process seems to have primarily the same genes for AUC 0.55-0.7 that are most enriched
    - It seems linked to ion channeling
- Feet and toe abnormalities seem to occur once or twice too
- Could this be an indication to genes that are related to very small molecules? (like steroids, ions, neuro-molecules?)
- Most significant: autosomal dominant inheritence
    - Very interesting plot, the HPO bokeh plot of n=10
- 

<br/><br/>
TODO: make word counts of genes and term names?


In [43]:
combined_df_ext = pd.DataFrame(columns=enrichr_dataset['BioPlanet_2019'].columns)
for key, value in enrichr_dataset.items():
    subdf = pd.DataFrame(columns=combined_df_ext.columns)
    for auc in value['AUC'].unique():
        subdf = subdf.append(value[value['AUC'] == auc], ignore_index=True)
    combined_df_ext = combined_df_ext.append(subdf, ignore_index=True)
combined_df_ext

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,3,Activation of TRKA receptors,NTRK2,1.0,BioPlanet_2019
1,0.5,4,MAP kinase pathway regulation through dual spe...,DUSP6,1.0,BioPlanet_2019
2,0.5,5,ERK inactivation,DUSP6,1.0,BioPlanet_2019
3,0.5,7,Nucleotide-like (purinergic) G-protein coupled...,P2RY12,1.0,BioPlanet_2019
4,0.5,8,Gap junction assembly,GJB3,1.0,BioPlanet_2019
...,...,...,...,...,...,...
3359,1.0,17,chr22,"SMARCB1, TMPRSS6, PDGFB, TCF20, PLA2G6, TYMP, ...",1.0,Chromosome_Location_hg19
3360,1.0,16,chr6 cox hap2,"TNXB, CYP21A2, TUBB, COL11A2, SKIV2L, VARS2, C...",1.0,Chromosome_Location_hg19
3361,1.0,18,chr21,"COL18A1, KCNE2, TMPRSS3, ITGB2, DYRK1A, TSPEAR...",1.0,Chromosome_Location_hg19
3362,1.0,20,chr3,"SETD5, SETD2, GFM1, ZBTB20, PLOD2, SLC6A1, MYL...",1.0,Chromosome_Location_hg19


In [44]:
combined_df_ext.where((combined_df_ext['AUC'] == 0.5) & (combined_df_ext['Source'] == 'BioPlanet_2019')).dropna()

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,3,Activation of TRKA receptors,NTRK2,1.0,BioPlanet_2019
1,0.5,4,MAP kinase pathway regulation through dual spe...,DUSP6,1.0,BioPlanet_2019
2,0.5,5,ERK inactivation,DUSP6,1.0,BioPlanet_2019
3,0.5,7,Nucleotide-like (purinergic) G-protein coupled...,P2RY12,1.0,BioPlanet_2019
4,0.5,8,Gap junction assembly,GJB3,1.0,BioPlanet_2019
5,0.5,9,Interleukin-11 pathway,DUSP6,1.0,BioPlanet_2019
6,0.5,10,ADP signalling through P2Y purinoceptor 12,P2RY12,1.0,BioPlanet_2019
7,0.5,11,Gap junction trafficking and regulation,GJB3,1.0,BioPlanet_2019
8,0.5,12,Nuclear events mediated by MAP kinases,DUSP6,1.0,BioPlanet_2019
9,0.5,13,Signal amplification,P2RY12,1.0,BioPlanet_2019


In [45]:
word_count_df = count_words_in_df(combined_df_ext)

In [46]:
nwkdf = pd.DataFrame(columns=word_count_df.columns)
for auc in word_count_df['auc'].unique():
    nwkdf = nwkdf.append(word_count_df[word_count_df['auc'] == auc][:5])
nwkdf

,word,count,auc
0,signaling,19,0.50
1,pathway,14,0.50
2,disease,9,0.50
3,syndrome,9,0.50
4,(bulk,9,0.50
498,signaling,13,0.55
499,pathway,10,0.55
500,disease,10,0.55
501,regulation,9,0.55
502,syndrome,9,0.55


In [47]:
plot = plot_count_results(nwkdf, item='word')
save(plot)

'/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/enrichr_word_count_bokeh.html'

In [48]:
gene_count_df = pd.DataFrame(columns=['gene', 'count', 'auc'])
for auc in combined_df_ext['AUC'].unique():
    subset_df = combined_df_ext[combined_df_ext['AUC'] == auc]
    gene_list = []
    unsplitted_gene_list = subset_df['Overlapping_Genes'].tolist()
    for unsplitted_gene in unsplitted_gene_list:
        gene_list += unsplitted_gene.split(', ')
    gene_count_dict = {}
    for gene in gene_list:
        if gene not in gene_count_dict.keys():
            gene_count_dict[gene] = 1
        else:
            gene_count_dict[gene] += 1
    temp_df = pd.DataFrame(columns=gene_count_df.columns)
    for key in gene_count_dict.keys():
        temp_temp_df = pd.DataFrame({'gene':key, 'count': gene_count_dict[key],
                                    'auc': auc}, index=[0])
        temp_df = temp_df.append(temp_temp_df, ignore_index=True)
    gene_count_df = gene_count_df.append(temp_df, ignore_index=True)

In [49]:
gene_count_df.sort_values(by=['auc','count'], ascending=[True, False], inplace=True)

In [50]:
ngkdf = pd.DataFrame(columns=gene_count_df.columns)
for auc in gene_count_df['auc'].unique():
    ngkdf = ngkdf.append(gene_count_df[gene_count_df['auc'] == auc][:5])

In [51]:
plot = plot_count_results(ngkdf, item='gene')
save(plot)

'/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/enrichr_gene_count_bokeh.html'

In [52]:
combined_df_ext = pd.DataFrame(columns=enrichr_dataset['BioPlanet_2019'].columns)
for key, value in enrichr_dataset.items():
    subdf = pd.DataFrame(columns=combined_df_ext.columns)
    for auc in value['AUC'].unique():
        subdf = subdf.append(value[value['AUC'] == auc], ignore_index=True)
    combined_df_ext = combined_df_ext.append(subdf, ignore_index=True)
combined_df_ext

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,3,Activation of TRKA receptors,NTRK2,1.0,BioPlanet_2019
1,0.5,4,MAP kinase pathway regulation through dual spe...,DUSP6,1.0,BioPlanet_2019
2,0.5,5,ERK inactivation,DUSP6,1.0,BioPlanet_2019
3,0.5,7,Nucleotide-like (purinergic) G-protein coupled...,P2RY12,1.0,BioPlanet_2019
4,0.5,8,Gap junction assembly,GJB3,1.0,BioPlanet_2019
...,...,...,...,...,...,...
3359,1.0,17,chr22,"SMARCB1, TMPRSS6, PDGFB, TCF20, PLA2G6, TYMP, ...",1.0,Chromosome_Location_hg19
3360,1.0,16,chr6 cox hap2,"TNXB, CYP21A2, TUBB, COL11A2, SKIV2L, VARS2, C...",1.0,Chromosome_Location_hg19
3361,1.0,18,chr21,"COL18A1, KCNE2, TMPRSS3, ITGB2, DYRK1A, TSPEAR...",1.0,Chromosome_Location_hg19
3362,1.0,20,chr3,"SETD5, SETD2, GFM1, ZBTB20, PLOD2, SLC6A1, MYL...",1.0,Chromosome_Location_hg19


In [53]:
combined_df_ext.where((combined_df_ext['AUC'] == 0.5) & (combined_df_ext['Source'] == 'BioPlanet_2019')).dropna()

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,3,Activation of TRKA receptors,NTRK2,1.0,BioPlanet_2019
1,0.5,4,MAP kinase pathway regulation through dual spe...,DUSP6,1.0,BioPlanet_2019
2,0.5,5,ERK inactivation,DUSP6,1.0,BioPlanet_2019
3,0.5,7,Nucleotide-like (purinergic) G-protein coupled...,P2RY12,1.0,BioPlanet_2019
4,0.5,8,Gap junction assembly,GJB3,1.0,BioPlanet_2019
5,0.5,9,Interleukin-11 pathway,DUSP6,1.0,BioPlanet_2019
6,0.5,10,ADP signalling through P2Y purinoceptor 12,P2RY12,1.0,BioPlanet_2019
7,0.5,11,Gap junction trafficking and regulation,GJB3,1.0,BioPlanet_2019
8,0.5,12,Nuclear events mediated by MAP kinases,DUSP6,1.0,BioPlanet_2019
9,0.5,13,Signal amplification,P2RY12,1.0,BioPlanet_2019


In [54]:
word_count_df = count_words_in_df(combined_df_ext)

In [55]:
nwkdf = pd.DataFrame(columns=word_count_df.columns)
for auc in word_count_df['auc'].unique():
    nwkdf = nwkdf.append(word_count_df[word_count_df['auc'] == auc][:5])
nwkdf

,word,count,auc
0,signaling,19,0.50
1,pathway,14,0.50
2,disease,9,0.50
3,syndrome,9,0.50
4,(bulk,9,0.50
498,signaling,13,0.55
499,pathway,10,0.55
500,disease,10,0.55
501,regulation,9,0.55
502,syndrome,9,0.55


In [56]:
plot = plot_count_results(nwkdf, item='word')
save(plot)

'/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/enrichr_word_count_bokeh.html'

In [57]:
gene_count_df = pd.DataFrame(columns=['gene', 'count', 'auc'])
for auc in combined_df_ext['AUC'].unique():
    subset_df = combined_df_ext[combined_df_ext['AUC'] == auc]
    gene_list = []
    unsplitted_gene_list = subset_df['Overlapping_Genes'].tolist()
    for unsplitted_gene in unsplitted_gene_list:
        gene_list += unsplitted_gene.split(', ')
    gene_count_dict = {}
    for gene in gene_list:
        if gene not in gene_count_dict.keys():
            gene_count_dict[gene] = 1
        else:
            gene_count_dict[gene] += 1
    temp_df = pd.DataFrame(columns=gene_count_df.columns)
    for key in gene_count_dict.keys():
        temp_temp_df = pd.DataFrame({'gene':key, 'count': gene_count_dict[key],
                                    'auc': auc}, index=[0])
        temp_df = temp_df.append(temp_temp_df, ignore_index=True)
    gene_count_df = gene_count_df.append(temp_df, ignore_index=True)

In [58]:
gene_count_df.sort_values(by=['auc','count'], ascending=[True, False], inplace=True)

In [59]:
ngkdf = pd.DataFrame(columns=gene_count_df.columns)
for auc in gene_count_df['auc'].unique():
    ngkdf = ngkdf.append(gene_count_df[gene_count_df['auc'] == auc][:5])

In [60]:
plot = plot_count_results(ngkdf, item='gene')
save(plot)

'/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/enrichr_gene_count_bokeh.html'

## Interpretation
It seems that a normal distribution like pattern is formed with a steeper slope after the maximum, starting at rougly AUC of 0.3 and ending at roughly 0.8.
<br/><br/>
- DisGeNET seems to outline an interesting picture with the top 10 term names:
    - For it seems that in the really bad performing genes the main cause could be infant development related. Later on, more neurological causes are identified.
        - Neurological seems clear to me... There is still so much unknown about the human mind.
        - Infant development related is a bit less clear, since there is a growing amount of research towards this infant development.
- GO_biological_process seems to have primarily the same genes for AUC 0.55-0.7 that are most enriched
    - It seems linked to ion channeling
- Feet and toe abnormalities seem to occur once or twice too
- Could this be an indication to genes that are related to very small molecules? (like steroids, ions, neuro-molecules?)
- Most significant: autosomal dominant inheritence
    - Very interesting plot, the HPO bokeh plot of n=10
- 

<br/><br/>
TODO: make word counts of genes and term names?


# Correlation analysis Enrichr results
Investigate the interactive legend.

- Bioplanet 2019:
    - Anticorrelation possible, high adjusted-pvalue (APV) when auc is low, but low APV when AUC is high.
    
- Wikipathways:
    - Shows simular anticorrelation to bioplanet, but peaks at 0.55 auc. Could be a normal distribution like.
    
- Kegg 2019:
    - Pretty much all over the place. Some significant when low AUC, but also some not significant. Seems like 2 different distributions are present. One distribution is a linear like from auc 0.5 apv 0.2 to auc 0.7 apv 0.6, but above that a normally distributed curve. Interesting, investigate futher.
    
- Elsevier:
    - Clear normal distribution like, with a peak at either 0.55 (everything is not significant) or 0.65 (largely not significant).

- GO Biological:
    - Similar to kegg, 2 distributions might be shown. A normal distribution like and an inverse exponent. Both seem to be crossing each other at auc 0.5 APV 0.4.

- HPO:
    - Is low and stays low. Tops at auc 0.55 and APV 0.15. Always significant.

- Jensen Diseases:
    - Not really following any distribution. Got some significant, some not and some in between. Tops at auc 0.55 and APV 1.
    
- PheWeb:
    - Exponentially correlated. "Low" when auc is low, get higher when AUC get higher up to auc 0.95 where it becomes more significant.
    
- Clinvar:
    - Multiple distributions present. One 'high' and one 'low'. Investigate futher.
    
- GWAS catalogue:
    - Maybe anti-correlated with some outliers?
    
- DisGeNet:
    - Always significant, simular to HPO.
    
- Omim Disease:
    - Anti-correlated with some points that are always significant.
    
- Omim expanded:
    - Not following omim disease pattern, more like a normal distribution that peaks at auc 0.7 APV 0.6.
    
- Human gene atlas:
    - Like a half cosinus wave with a very sudden drop at auc 0.8.
    
- ARCHS4 tissues:
    - Normal distribution like that tops over APV of 1 at AUC 0.65.
    
- HMDB Metabolites:
    - Says insigificant up untill auc 0.75, where it drops steeply.
    
- Chromosome location:
    - Always insigificant untill AUC 0.85, where chromosome X is always significant.
    
    
# AUC cutoffs where there is still some variation

- Bioplanet: 0.85

- Wikipathways: 0.85

- Kegg: 0.85

- Elsevier pathway collection: 0.80

- GO Biological: 0.80

- HPO: 0.75

- Jensen DISEASES: 0.80

- PheWEB: 0.7 (after that some points become more significant again, but that is commonly seen throughout every database)

- ClinVar: 0.85

- GWAS Catalogue: 0.9

- DisGeNET: 0.7

- OMIM_Disease: 0.85

- OMIM_Expanded: 0.80

- Human Gene Atlas: 0.95

- ARCHS4 tissues: 0.9

- HMDB Metabolites: 0.9

- Chromosome location: Odd one out (always insignificant except for auc 0.9-1.0 where chromosome X becomes significant)


 


In [65]:
combined_df_ext.head()

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,3,Activation of TRKA receptors,NTRK2,1.0,BioPlanet_2019
1,0.5,4,MAP kinase pathway regulation through dual spe...,DUSP6,1.0,BioPlanet_2019
2,0.5,5,ERK inactivation,DUSP6,1.0,BioPlanet_2019
3,0.5,7,Nucleotide-like (purinergic) G-protein coupled...,P2RY12,1.0,BioPlanet_2019
4,0.5,8,Gap junction assembly,GJB3,1.0,BioPlanet_2019


In [64]:
for source in combined_df_ext['Source'].unique():
    subset = combined_df_ext[combined_df_ext['Source'] == source]
    subset = subset.groupby(by='AUC').mean()
    rho, pval = stats.spearmanr(subset.index, subset['Adjusted_P-value'])
    print(f'Correlation coefficient for {source} is:{rho}')
    print(f'P-value of {source} is: {pval}')
    if pval >= 0.05:
        print(f'Fail to reject null hypothesis of {source} being uncorrelated, meaning they are uncorrelated.\n')
    else:
        print(f'Able to reject null hypothesis of {source} being uncorrelated, there is a:')
        if rho > 0:
            print('positive correlation. \n')
        else:
            print('negative correlation. \n')

Correlation coefficient for BioPlanet_2019 is:-0.8991204081865278
P-value of BioPlanet_2019 is: 0.0001661839752544972
Able to reject null hypothesis of BioPlanet_2019 being uncorrelated, there is a:
negative correlation. 

Correlation coefficient for WikiPathways_2019_Human is:-0.6885583156948685
P-value of WikiPathways_2019_Human is: 0.019136081775021636
Able to reject null hypothesis of WikiPathways_2019_Human being uncorrelated, there is a:
negative correlation. 

Correlation coefficient for KEGG_2019_Human is:-0.6102160571171791
P-value of KEGG_2019_Human is: 0.046180632934565924
Able to reject null hypothesis of KEGG_2019_Human being uncorrelated, there is a:
negative correlation. 

Correlation coefficient for Elsevier_Pathway_Collection is:-0.9339431975353206
P-value of Elsevier_Pathway_Collection is: 2.6028693621417825e-05
Able to reject null hypothesis of Elsevier_Pathway_Collection being uncorrelated, there is a:
negative correlation. 

Correlation coefficient for GO_Biologica

In [77]:
for source in combined_df_ext['Source'].unique():
    subset = combined_df_ext[combined_df_ext['Source'] == source]
    subset = subset.groupby(by='AUC').mean()
    rho, pval = stats.spearmanr(subset.index, subset['Adjusted_P-value'])
    print(subset)
    break
    print(f'Correlation coefficient for {source} is:{rho}')
    print(f'P-value of {source} is: {pval}')
    if pval >= 0.05:
        print(f'Fail to reject null hypothesis of {source} being uncorrelated, meaning they are uncorrelated.\n')
    else:
        print(f'Able to reject null hypothesis of {source} being uncorrelated, there is a:')
        if rho > 0:
            print('positive correlation. \n')
        else:
            print('negative correlation. \n')

      Adjusted_P-value
AUC                   
0.50      1.000000e+00
0.55      1.000000e+00
0.60      1.000000e+00
0.65      9.580241e-01
0.70      9.615924e-01
0.75      9.699334e-01
0.80      9.860955e-01
0.85      5.848485e-01
0.90      1.880091e-01
0.95      2.542807e-02
1.00      2.557615e-12


In [75]:
combined_df_ext

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,3,Activation of TRKA receptors,NTRK2,1.0,BioPlanet_2019
1,0.5,4,MAP kinase pathway regulation through dual spe...,DUSP6,1.0,BioPlanet_2019
2,0.5,5,ERK inactivation,DUSP6,1.0,BioPlanet_2019
3,0.5,7,Nucleotide-like (purinergic) G-protein coupled...,P2RY12,1.0,BioPlanet_2019
4,0.5,8,Gap junction assembly,GJB3,1.0,BioPlanet_2019
...,...,...,...,...,...,...
3359,1.0,17,chr22,"SMARCB1, TMPRSS6, PDGFB, TCF20, PLA2G6, TYMP, ...",1.0,Chromosome_Location_hg19
3360,1.0,16,chr6 cox hap2,"TNXB, CYP21A2, TUBB, COL11A2, SKIV2L, VARS2, C...",1.0,Chromosome_Location_hg19
3361,1.0,18,chr21,"COL18A1, KCNE2, TMPRSS3, ITGB2, DYRK1A, TSPEAR...",1.0,Chromosome_Location_hg19
3362,1.0,20,chr3,"SETD5, SETD2, GFM1, ZBTB20, PLOD2, SLC6A1, MYL...",1.0,Chromosome_Location_hg19


In [81]:
for term in combined_df_ext['Term_Name'].unique():
    subset = combined_df_ext[combined_df_ext['Term_Name'] == term]
    min_thres = subset['AUC'].min()
    max_thres = subset['AUC'].max()
    if min_thres == max_thres:
        continue
    data_source = subset['Source'].unique()[0]
    rho, pval = stats.spearmanr(subset['AUC'], subset['Adjusted_P-value'])
    if pval <= 0.05:
        if rho > 0:
            print(f"Source {data_source} has a significant positive correlation:")
            print(f"Term: {term}")
            print(f"p-value: {pval}")
            print(f"correlation coefficient: {rho}")
            print(f"Running from {min_thres} to {max_thres}")
            print("\n")

Source Jensen_DISEASES has a significant positive correlation:
Term: Ollier disease
p-value: 0.0
correlation coefficient: 1.0
Running from 0.75 to 0.85




In [46]:
pheweb = combined_df_ext.where((combined_df_ext['AUC'] <= 0.9) & (combined_df_ext['Source'] == 'PheWeb_2019')).dropna()

In [47]:
pheweb.index = pheweb.reindex(labels=np.arange(pheweb.shape[0]), axis=0).index
grouped_pheweb = pheweb.groupby(by='AUC').mean()
grouped_pheweb
rho, pval = stats.spearmanr(grouped_pheweb.index, grouped_pheweb['Adjusted_P-value'])
print(f'Correlation coefficient for pheweb is:{rho}')
print(f'P-value of pheweb is: {pval}')
if pval >= 0.05:
    print(f'Fail to reject null hypothesis of pheweb being uncorrelated, meaning they are uncorrelated.\n')
else:
    print(f'Able to reject null hypothesis of pheweb being uncorrelated, there is a:')
    if rho > 0:
        print('positive correlation. \n')
    else:
        print('negative correlation. \n')

Correlation coefficient for pheweb is:0.45643546458763845
P-value of pheweb is: 0.2168375456189006
Fail to reject null hypothesis of pheweb being uncorrelated, meaning they are uncorrelated.



In [48]:
# Both 0.7 and 0.9 state that PheWeb is positively correlated when AUC cutoff is left at 0.9.

In [49]:
kegg = combined_df_ext.where((combined_df_ext['AUC'] <= 0.85) & (combined_df_ext['Source'] == 'KEGG_2019_Human')).dropna()

In [50]:
kegg.index = kegg.reindex(labels=np.arange(kegg.shape[0]), axis=0).index
grouped_kegg = kegg.groupby(by='AUC').mean()
grouped_kegg
rho, pval = stats.spearmanr(grouped_kegg.index, grouped_kegg['Adjusted_P-value'])
print(f'Correlation coefficient for kegg is:{rho}')
print(f'P-value of kegg is: {pval}')
if pval >= 0.05:
    print(f'Fail to reject null hypothesis of kegg being uncorrelated, meaning they are uncorrelated.\n')
else:
    print(f'Able to reject null hypothesis of kegg being uncorrelated, there is a:')
    if rho > 0:
        print('positive correlation. \n')
    else:
        print('negative correlation. \n')

Correlation coefficient for kegg is:0.10910894511799618
P-value of kegg is: 0.7970385741881917
Fail to reject null hypothesis of kegg being uncorrelated, meaning they are uncorrelated.



In [51]:
# Further confirms that KEGG might have multiple distributions.

In [52]:
jensen_d = combined_df_ext.where((combined_df_ext['AUC'] <= 0.8) & (combined_df_ext['Source'] == 'Jensen_DISEASES')).dropna()

In [53]:
jensen_d.index = jensen_d.reindex(labels=np.arange(jensen_d.shape[0]), axis=0).index
grouped_jensen_d = jensen_d.groupby(by='AUC').mean()
rho, pval = stats.spearmanr(grouped_jensen_d.index, grouped_jensen_d['Adjusted_P-value'])
print(f'Correlation coefficient for jensen_d is:{rho}')
print(f'P-value of jensen_d is: {pval}')
if pval >= 0.05:
    print(f'Fail to reject null hypothesis of jensen_d being uncorrelated, meaning they are uncorrelated.\n')
else:
    print(f'Able to reject null hypothesis of jensen_d being uncorrelated, there is a:')
    if rho > 0:
        print('positive correlation. \n')
    else:
        print('negative correlation. \n')

Correlation coefficient for jensen_d is:-0.642857142857143
P-value of jensen_d is: 0.11939237342741094
Fail to reject null hypothesis of jensen_d being uncorrelated, meaning they are uncorrelated.



- Look into pathways in the databases themselfs, not the complete database.
    - Final enrichr analysis, investigate best hits, are these significant?
    
        - Is there something here?
        
            -No: No specific enrichment for any pathways.
            
            -Yes: Very good! Investigate further.
            
- Investigate CAPICE performance on UMCG gene panels.

- Train new model on umcg gene panels.
    - 1st option: Top 3 worst and top 3 best by Fingerspitsengefuhl (then put into discussion that more might have been better)
    - 2nd option: Use bad performing genes as panel to train on.
        - Slice out problem genes or genes that share some commonalities?
    - First disease / gene panel specific model training without adding features.
    - Then add new features.



In [54]:
nlines = 0
temp_df = pd.DataFrame(columns=combined_df_ext.columns)
for pathway in combined_df_ext['Term_Name'].unique():
    nlines += 1
    subset = combined_df_ext[combined_df_ext['Term_Name'] == pathway]
    x = subset['AUC']
    y = subset['Adjusted_P-value']
    rho, pval = stats.spearmanr(x, y)
    if pval < 0.05 and rho > 0:
        temp_df = temp_df.append(subset, ignore_index=True)
second_word_count = count_words_in_df(temp_df)
second_word_count.sort_values(by='count', ascending=False).head(10)

/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/scipy/stats/stats.py:4196: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


,word,count,auc
0,ollier,1,0.75
1,disease,1,0.75
2,ollier,1,0.80
3,disease,1,0.80
4,ollier,1,0.85
5,disease,1,0.85


In [55]:
combined_df_ext['n_genes'] = combined_df_ext['Overlapping_Genes'].apply(lambda x: split_and_count_words(x))

In [56]:
nlines = 0
for pathway in combined_df_ext['Term_Name'].unique():
    nlines += 1
    subset = combined_df_ext[combined_df_ext['Term_Name'] == pathway]
    x = subset['AUC']
    y = subset['Adjusted_P-value']
    min_genes = subset['n_genes'].min()
    max_genes = subset['n_genes'].max()
    rho, pval = stats.spearmanr(x, y)
    if pval < 0.1 and rho > 0:
        genes = subset['Overlapping_Genes'].values.tolist()
        output_genes = []
        for gene in genes:
            intermediate_genes = split_and_count_words(gene, return_value = None)
            for g in intermediate_genes:
                if g not in output_genes:
                    output_genes.append(g)
        if len(output_genes) >= 3:
            output_genes = ', '.join(output_genes)
            print(f'Able to reject null hypothesis of {pathway} for AUC: '
                  f'{x.values[0]} - {x.values[-1]} and source: '
                  f'{subset["Source"].unique()[0]} for genes: {output_genes}')
            print('\n')

/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/scipy/stats/stats.py:4196: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


In [57]:
interesting_pathway = ['Transfer RNA aminoacylation',
                        'Oxytocin signaling pathway',
                        'Tinnitus',
                        'Acne',
                        'COLONIC MUCOSA'
                      ]
for pathway in interesting_pathway:
    print(pathway)
    print(combined_df_ext[combined_df_ext['Term_Name'] == pathway])
    print('\n')

Transfer RNA aminoacylation
      AUC Rank                    Term_Name Overlapping_Genes  \
108  0.75    3  Transfer RNA aminoacylation        YARS, GARS   

     Adjusted_P-value          Source  n_genes  
108               1.0  BioPlanet_2019        2  


Oxytocin signaling pathway
Empty DataFrame
Columns: [AUC, Rank, Term_Name, Overlapping_Genes, Adjusted_P-value, Source, n_genes]
Index: []


Tinnitus
       AUC Rank Term_Name     Overlapping_Genes  Adjusted_P-value  \
1658  0.90   11  Tinnitus          TRPC6, AGBL1               1.0   
1668  0.95    2  Tinnitus  CACNB2, TRPC6, AGBL1               1.0   

           Source  n_genes  
1658  PheWeb_2019        2  
1668  PheWeb_2019        3  


Acne
       AUC Rank Term_Name Overlapping_Genes  Adjusted_P-value       Source  \
1583  0.70   18      Acne             NRXN3               1.0  PheWeb_2019   
1605  0.75   20      Acne             NRXN3               1.0  PheWeb_2019   

      n_genes  
1583        1  
1605        1  


COLO

In [62]:
nlines = 0
for pathway in combined_df_ext['Term_Name'].unique():
    nlines += 1
    subset = combined_df_ext[combined_df_ext['Term_Name'] == pathway]
    x = subset['AUC']
    y = subset['Adjusted_P-value']
    min_genes = subset['n_genes'].min()
    max_genes = subset['n_genes'].max()
    rho, pval = stats.spearmanr(x, y)
    if pval < 0.05 and rho > 0:
        genes = subset['Overlapping_Genes'].values.tolist()
        output_genes = []
        for gene in genes:
            intermediate_genes = split_and_count_words(gene, return_value = None)
            for g in intermediate_genes:
                if g not in output_genes:
                    output_genes.append(g)
        if len(output_genes) >= 3:
            output_genes = ', '.join(output_genes)
            print(f'Able to reject null hypothesis of {pathway} for AUC: '
                  f'{x.values[0]} - {x.values[-1]} and source: '
                  f'{subset["Source"].unique()[0]} for genes: {output_genes}')
            print('\n')

/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/scipy/stats/stats.py:4196: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


In [59]:
combined_df_ext[combined_df_ext['Term_Name'] == 'Tinnitus']

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source,n_genes
1658,0.90,11,Tinnitus,"TRPC6, AGBL1",1.0,PheWeb_2019,2
1668,0.95,2,Tinnitus,"CACNB2, TRPC6, AGBL1",1.0,PheWeb_2019,3


In [60]:
combined_df_ext[combined_df_ext['Term_Name'].str.lower().str.startswith('brugada')]

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source,n_genes
1846,0.95,19,brugada syndrome,"CACNB2, CACNA1C",0.078511,ClinVar_2019,2


- Genes involved in identifying brugada syndrome might be troublesome for CAPICE, but then again, if we look at the command 'combined_df_ext[combined_df_ext['Term_Name'].str.lower().str.startswith('brugada')]' there's not a lot of genes in that.
- Colonic Mucosa is not really that interesting either. Starts at AUC 0.65 and ends at 0.75, only increasing slightly with an adjusted p-value of at least 0.999925 (very insignificant).
    - Transfer RNA aminoacylation is a bit different, still insignificant but starts at 0.594633.
    - Same goes for Oxytocin signaling pathway, starts at 0.941531
    - Tinnitus is a bit different, only has 2 genes in the lower AUC values but starts out at 0.218952 and goes up to 1 from auc 0.5-0.9.
    
    
- When a pad performing gene panels is found, make a decision tree on what to do next.
- Keep in mind, I have 4 times the amount of benign samples than malignant samples.
- Chi squared / regression correlation analysis between false positive / negative samples on a shared feature.
- Turning in concept verion paper on 6th of July is possible with the 2 weeks deadline for feedback. 
- All methods to seperate python files and then import them to clean up notebook.
- Phase 2 will produce graphs, order them to the story I want to tell.
    - Thesis is a bit more than a paper, but a bit less than a bachelor

In [61]:
umcg_genepanels = pd.read_excel('/home/rjsietsma/Documents/LAB-F0701 Excel van Genenlijst panels Genoomdiagnostiek UMCG_versie 4 mei.xlsx', sheet_name=None)

FileNotFoundError: [Errno 2] No such file or directory: '/home/rjsietsma/Documents/LAB-F0701 Excel van Genenlijst panels Genoomdiagnostiek UMCG_versie 4 mei.xlsx'

In [ ]:
start = '5GPM'
output_genepanels = {}
in_genepanels = False
for value in umcg_genepanels.values():
    subject = value.columns[0]
    if subject == start:
        in_genepanels = True
    if in_genepanels:
        specific_disease = value.iloc[0,0]
        if isinstance(specific_disease, float):
            specific_disease = subject
        first_col = subject
        real_col = 'Gen'
        gen_index = value[value[first_col] == real_col].index
        if gen_index.shape[0] < 1:
            continue
        if value.shape[1] > 1:
            value = value.iloc[:,0]
        value = value.shift(-(gen_index.values[0] + 1))
        if not isinstance(value, pd.DataFrame):
            value = pd.DataFrame(value)
        value.columns = [real_col]
        value.replace('Terug naar inhoudsopgave', np.NaN, inplace=True)     
        value.dropna(inplace=True)
        output = value.values.tolist()
        real_output = []
        for i in output:
            real_output = real_output + i
        if subject not in output_genepanels.keys():
            temp = {specific_disease: real_output}
            output_genepanels[subject] = temp
        else:
            temp = {specific_disease: real_output}
            subject_dict = output_genepanels[subject]
            subject_dict.update(temp)

output_genepanels

In [ ]:
auc_data = data[['gene','default_auc']].copy()
auc_data.columns = ['gene','auc']

In [ ]:
auc_data.set_index('gene', inplace=True)

In [ ]:
auc_data

In [ ]:
x = auc_data.index.values
x2 = auc_data['auc'].values
auc_data_series = pd.Series(x2, index=x)

In [ ]:
auc_data_dict = auc_data_series.to_dict()

In [ ]:
umcg_genepanel_analysis = pd.DataFrame(columns=['category','panel','min_auc',
                                                'max_auc','avg_auc',
                                                'std','med_auc','used%','used_n',
                                                'genes_no_data'])
for key in output_genepanels.keys():
    for key2, value in output_genepanels[key].items():
        disease = ", ".join([key, key2])
        n_genes_tot = len(value)
        genes_no_auc_data = []
        keep_track = np.full(shape=len(value), fill_value=np.NaN)
        for iteration, gene in enumerate(value):
            if gene in auc_data_dict.keys():
                keep_track[iteration] = auc_data_dict[gene]
            else:
                genes_no_auc_data.append(gene)
        keep_track = keep_track[~np.isnan(keep_track)]
        min_auc = keep_track.min()
        max_auc = keep_track.max()
        average_auc = keep_track.mean()
        std_auc = keep_track.std()
        median_auc = np.median(keep_track)
        n_used = keep_track.size
        used_n = f'{n_used}/{n_genes_tot}'
        p_used = round(n_used / n_genes_tot * 100)
        x = pd.DataFrame({'category':[key],
                          'panel':[key2],
                          'min_auc':[min_auc],
                         'max_auc':[max_auc],
                         'avg_auc':[average_auc],
                         'std':[std_auc],
                         'med_auc':[median_auc],
                         'used%':[p_used],
                         'used_n':[used_n],
                         'genes_no_data':[genes_no_auc_data]}
                         , index=[1])
        umcg_genepanel_analysis = umcg_genepanel_analysis.append(x, ignore_index=True)
umcg_genepanel_analysis           

In [ ]:
umcg_genepanel_analysis.sort_values(by='avg_auc', ascending=True).tail(10)

In [ ]:
categories = umcg_genepanel_analysis['category'].unique()
pallette = inferno(len(categories) + 1)
colormap = dict(zip(categories, pallette))
umcg_genepanel_analysis['color'] = umcg_genepanel_analysis['category'].map(colormap)

In [ ]:
umcg_genepanel_analysis.sort_values(by='avg_auc', inplace=True)
umcg_genepanel_analysis['placeholder_x'] = np.arange(umcg_genepanel_analysis.shape[0])

In [ ]:
placeholder_actual_dict = dict(zip(umcg_genepanel_analysis['placeholder_x'].tolist(),
                                  umcg_genepanel_analysis['panel'].tolist()))

In [ ]:
output_file(filename='./umcg_genepanel_barplot.html',
            title='Boxplot UMCG Genepanels'
           )
title = 'Boxplot of UMCG genepanels'
source = ColumnDataSource(umcg_genepanel_analysis)
p = figure(title=title,
           plot_width=1500,
               plot_height=600,
               toolbar_location='right',
               toolbar_sticky=False)
p.add_layout(Legend(), 'right')
p.vbar(x='placeholder_x',
      top='avg_auc',
      color='color',
      legend_field='category',
      width=0.9,
      source=source)
p.add_tools(HoverTool(tooltips=[
    ('Category','@category'),
    ('Panel','@panel'),
    ('AUC','@avg_auc')
]))
save(p)


In [ ]:
umcg_genepanel_analysis[umcg_genepanel_analysis['category'] == 'Primaire Immuundeficiëntie']

In [ ]:
umcg_genepanel_analysis.sort_values(by='avg_auc', ascending=True, inplace=True)
umcg_genepanel_analysis[umcg_genepanel_analysis['panel'].str.lower().str.startswith('dyslipid')]

In [ ]:
wilcoxon_analysis = pd.DataFrame(columns=['category', 'compared_to',
                                          'p_two_sided', 'p_lower',
                                          'p_greater'])
wilcoxon_alternatives = ['two-sided', 'greater', 'less']
for category in umcg_genepanel_analysis['category'].unique():
    subset = umcg_genepanel_analysis[umcg_genepanel_analysis['category'] == category]
    subset_auc = np.array(subset['avg_auc'])
    not_subset = umcg_genepanel_analysis[umcg_genepanel_analysis['category'] != category]
    for n_sub in not_subset['category'].unique():
        two = None
        greater = None
        less = None
        sub_subset = not_subset[not_subset['category'] == n_sub]
        sub_subauc = np.array(sub_subset['avg_auc'])
        for alt in wilcoxon_alternatives:
            _, p = stats.mannwhitneyu(subset_auc, sub_subauc, alternative=alt)
            if alt == 'two-sided':
                two = p
            elif alt == 'greater':
                greater = p
            elif alt == 'less':
                less = p
        sub_cat_df = pd.DataFrame(
            {
                'category':category,
                'compared_to':n_sub,
                'p_two_sided':two,
                'p_lower':less,
                'p_greater':greater
            }, index=[0]
        )
        wilcoxon_analysis = wilcoxon_analysis.append(sub_cat_df, ignore_index=True)
        
        

In [ ]:
categories = wilcoxon_analysis['category'].unique()
pallette = inferno(len(categories) + 1)
colormap = dict(zip(categories, pallette))
wilcoxon_analysis['color'] = wilcoxon_analysis['compared_to'].map(colormap)

In [ ]:
to_be_placeholded = wilcoxon_analysis['compared_to'].unique()
placehold_array = np.arange(len(to_be_placeholded))
placeholder_dict = dict(zip(to_be_placeholded, placehold_array))
wilcoxon_analysis['x_placeholder'] = wilcoxon_analysis['compared_to'].map(placeholder_dict)

In [ ]:
wilcoxon_analysis[wilcoxon_analysis['category'] == 'Hart- en vaatziekten']

In [ ]:
x = np.random.randint(low=20,high=40,size=(20))
y = np.random.randint(20,size=(20))
stats.mannwhitneyu(x,y, alternative='greater')

In [ ]:
for category in wilcoxon_analysis['category'].unique():
    subset = wilcoxon_analysis[wilcoxon_analysis['category'] == category]
    ax = subset.plot.bar(x='compared_to',y=['p_two_sided', 'p_lower', 'p_greater'],
                         colormap='viridis',
                         figsize=(10,5),
                         title=f'Mann-Whitney analysis of {category}'
                        )
    ax.hlines(y=0.05,xmin=-10,xmax=100)
    ax.set_ylabel('P-value')
    ax.set_xlabel('Panel')

Now what do the plots above mean?

    - P_two_sided tests the null hypothesis that the distribution of title is different from the alternative. So when P is low, reject H0 (meaning there is a statisticial difference).
    - P_lower tests the alternative hypothesis that the median difference is negative. When P is low, reject H0, meaning that the median difference is negative. (If this value is low, it is statistically proven that x is lower than y)
    - P_higher tests the alternative hypothesis that the median difference is positive. When P is low, reject h0, meaning that the median difference is positive. (If this value is low, it is statistically proven that x is higher than y)

# From the Wilcoxon / Mannwhitney analysis
##### Hart- en vaatziekten
Every p_greater has a high P-value, indicating the category is not performing better than any other panel. Interestingly, Amyloidose, Leukemie-Lymfoom, 5GPM, OWA, Noonan syndroom, Cardiomyopathie bij kinderen, mitochondriele aandoeningen, hyper/hypofosfatemie, primaire cilliare dyskinesie en angio-oedeem also accepts H0 for two-sided and less, so it performs simular to them.
##### Erfelijke Kanker
Now quite the opposite of Hart- en vaatziekten, where p_lower accepts H0 except for maybe Angio-oedeem, meaning that everything is performing either worse or on par. Only Hart- en vaatziekten, epilepsie, metabole & leverziekten, aangeboren hartafwijkingen have some statistical proof of being better / worse than erfelijke kanker.
##### Primaire Immuundeficientie
Primaire immuundefientie is pretty much the average panel, only Hart- en vaatziekten performs statistically performs worse. Everything else is null hypothesis.
###### Angio-oedeem
Should be performing the best, a lot of high less p values is seen. But the null hypothesis for two-sided and greater can't be rejected in a single instance, so it might be performing better at sight, but it performs just average (which is good, this means CAPICE has a high average in the UMCG genepanels).


In [ ]:
allvs1 = pd.DataFrame(columns=['two-sided', 'less', 'greater'])
for category in umcg_genepanel_analysis['category'].unique():
    if category == '5GPM':
        continue
    x = np.array(umcg_genepanel_analysis[umcg_genepanel_analysis['category'] == category]['avg_auc'])

    y = np.array(umcg_genepanel_analysis.where(
        ~(umcg_genepanel_analysis['category'] == category) & 
        ~(umcg_genepanel_analysis['category'] == '5GPM')
    ).dropna()['avg_auc'])
    output = {'category':'all',
              'compared_to':category,
              'two-sided':None,
              'less':None,
              'greater':None}
    for alternative in wilcoxon_alternatives:
        _, p = stats.mannwhitneyu(x, y, alternative=alternative)
        output[alternative] = p
    allvs1 = allvs1.append(
        pd.DataFrame(output, index=[0]), ignore_index=True
    )
allvs1        

In [ ]:
allvs1.to_csv('./not_saving_directory/previousmodelauc.csv', index=False)

In [ ]:
ax = allvs1.plot(y=['two-sided','less','greater'], x='compared_to', kind='bar', colormap='viridis', figsize=(10,5),
                title='Mann-Whitney analysis of panel vs all.')
ax.hlines(y=0.05,xmin=-10,xmax=100)
ax.set_ylabel('P-value')
ax.set_xlabel('Panel')

In [ ]:
umcg_genepanel_analysis.where(~(umcg_genepanel_analysis['category'] == 'Hart- en vaatziekten') & 
                             ~(umcg_genepanel_analysis['category'] == '5GPM')).dropna()

In [ ]:
havd = output_genepanels['Hart- en vaatziekten']

In [ ]:
lohavdg = [] # List Of Heart And Vascular Disease Genes
for key in havd.keys():
    if key.lower().startswith('dyslipid'):
        lohavdg = lohavdg + havd[key]
lohavdg = list(dict.fromkeys(lohavdg))

In [ ]:
n_dislipid = pd.DataFrame(columns=['total', 'benign', 'malign'])
counting_dict = {
    'total':0,
    'benign':0,
    'malignant':0
}
for gene in lohavdg:
    subset = data[data['gene'] == gene]
    if subset.shape[0] > 0:
        counting_dict['total'] += subset['n_tot'].values[0]
        counting_dict['benign'] += subset['n_benign'].values[0]
        counting_dict['malignant'] += subset['n_malign'].values[0]
print(f"In dyslipidimie related genes, a total of {counting_dict['total']} variants are present, "
      f"of which {counting_dict['benign']} are benign and {counting_dict['malignant']} are malignant.")

In [ ]:
data

# Preparing ClinVar and VKGL databases
What genome build does VKGL have?
    - 37


In [ ]:
def get_clinvar_header(clinvar_loc):
    header = None
    with gzip.open(clinvar_loc) as clinvar_file:
        while True:
            line = clinvar_file.readline().decode('utf-8')
            if line.startswith('#CHROM'):
                header = line.strip().split('\t')
                break
    return header
clinvar_loc = '/run/media/rjsietsma/evo2tb/linux/Datafiles/clinvar.vcf.gz'
header = get_clinvar_header(clinvar_loc)
clinvar = pd.read_csv(clinvar_loc, comment='#', names=header, sep='\t', compression='gzip')
clinvar = clinvar[~clinvar['INFO'].str.lower().str.contains('clnsig=uncertain')]
clinvar = clinvar.reset_index().drop('index', axis=1)
clinvar

In [ ]:
clinvar.loc[0,'INFO'].split(';')

In [ ]:
clinvar.loc[0,:].str.contains('benign')['INFO']

In [ ]:
def clinvar_extract_clnsig(x):
    return_value = np.NaN
    info_list = x.split(';')
    for y in info_list:
        if y.startswith('CLNSIG'):
            label = y.split('=')[1].lower()
            if (label.find('benign') != -1):
                return_value = 'LB'
            elif (label.find('pathogenic') != -1):
                return_value = 'LP'
    return return_value

In [ ]:
clinvar['label'] = clinvar['INFO'].apply(lambda x: clinvar_extract_clnsig(x))
clinvar.dropna(axis=0, subset=['label'], inplace=True)
clinvar = clinvar.reset_index().drop('index', axis=1)
clinvar

In [ ]:
vkgl = pd.read_csv('/run/media/rjsietsma/evo2tb/linux/Datafiles/VKGL_public_consensus_jun2020.csv.gz', compression='gzip')
vkgl = vkgl[vkgl['classification'] != 'VUS']

In [ ]:
vkgl.drop_duplicates(subset=['chromosome','start','ref','alt'], inplace=True)
vkgl = vkgl.reset_index().drop('index',axis=1)[['chromosome','start','ref','alt','classification']]
vkgl_cols = ['#CHROM','POS','REF','ALT', 'label']
vkgl.columns = vkgl_cols
vkgl['INFO'] = '.'

In [ ]:
clinvar_vkgl = clinvar.append(vkgl, ignore_index=True).drop(['ID','QUAL','FILTER'], axis=1)

In [ ]:
def build_nondupe_clinvarvkgl(x):
    dupe_check = vkgl_cols[:-1]
    label_check = vkgl_cols
    if (vkgl[vkgl_cols] == x[vkgl_cols]).all(axis=1).sum() > 0:
        print(x)